In [1]:
!gdown 1O4YR4UBatOLnaP4gMHbmFw7UJvhhxFwq
!gdown 1-7aMdKW4KcCKLwoUKC3XxdIwfIKkzwx6
!unzip -q train_data.zip
!unzip -q test_data.zip

Downloading...
From: https://drive.google.com/uc?id=1O4YR4UBatOLnaP4gMHbmFw7UJvhhxFwq
To: /content/train_data.zip
100% 173M/173M [00:05<00:00, 30.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-7aMdKW4KcCKLwoUKC3XxdIwfIKkzwx6
To: /content/test_data.zip
100% 73.0M/73.0M [00:02<00:00, 25.1MB/s]


In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import GlobalAveragePooling2D, Dense, BatchNormalization
from keras.applications import EfficientNetV2L
from keras.callbacks import EarlyStopping

data_dir = '/content/train_data'
input_size = (480, 480)

train_datagen = ImageDataGenerator(vertical_flip=True,
                                   horizontal_flip=True,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   validation_split=0.2)

valid_datagen = ImageDataGenerator(validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
        data_dir,
        target_size=input_size,
        batch_size=64,
        class_mode='categorical',
        subset='training',
        seed=22)

validation_generator = valid_datagen.flow_from_directory(
        data_dir,
        target_size=input_size,
        batch_size=64,
        class_mode='categorical',
        subset='validation',
        seed=22)

# Load the pre-trained EfficientNetV2 model
base_model = EfficientNetV2L(include_top=False, weights='imagenet', input_shape=(480, 480, 3))

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(256, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor = 'val_accuracy',
                               patience=3,
                               restore_best_weights=True)

model.fit(train_generator, epochs=10, validation_data=validation_generator, callbacks=[early_stopping])

#### Predictions on test data

In [ ]:
from PIL import Image
import os
import numpy as np
from tqdm import tqdm

def read_and_resize_images(folder_path):
    image_files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

    resized_images = np.empty((len(image_files), 480, 480, 3), dtype=np.uint8)

    for i, image_file in tqdm(enumerate(image_files)):
        image_path = os.path.join(folder_path, image_file)

        img = Image.open(image_path)

        img = img.resize((480, 480))

        resized_images[i] = np.array(img)

    return resized_images

folder_path = '/kaggle/working/test_data'
resized_images_array = read_and_resize_images(folder_path)

In [ ]:
preds = model.predict(resized_images_array)
preds = np.argmax(preds, axis=1)
preds += 1

In [ ]:
import os
import pandas as pd


file_names = os.listdir('/kaggle/working/test_data')

df = pd.DataFrame({'name': file_names})
df['predicted'] = preds

df.to_csv('q1_submission.csv', index=False)